<a href="https://colab.research.google.com/github/thatwhoknows/iitg_sa/blob/main/Final_Hackathon_Submission_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Set-Up

In [1]:
# Install and import the required libraries
!pip install pathway
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.
import pandas as pd
import numpy as np
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
import time
from datetime import datetime, timedelta
import math

In [2]:
# load and preview data

# files.upload()
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset.csv', index_col='ID')
df.head()

,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
ID,,,,,,,,,,,
0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


# 1. Pre-Processing

In [3]:
# @title Set numerical and categorical columns
cat_cols=['SystemCodeNumber', 'VehicleType', 'TrafficConditionNearby', 'IsSpecialDay', 'Latitude', 'Longitude'] # Set categorical columns
cols_to_encode = ['SystemCodeNumber', 'VehicleType', 'TrafficConditionNearby'] # To be encoded
num_cols = ['Capacity', 'Occupancy', 'QueueLength'] # Set numerical columns
date_time_cols = ['LastUpdatedDate', 'LastUpdatedTime'] # Date and time (string) columns

# Check which columns are under what:
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)
print("To Encode:")
print(cols_to_encode)
print('Date/time:')
print(date_time_cols)

Categorical Variables:
['SystemCodeNumber', 'VehicleType', 'TrafficConditionNearby', 'IsSpecialDay', 'Latitude', 'Longitude']
Numerical Variables:
['Capacity', 'Occupancy', 'QueueLength']
To Encode:
['SystemCodeNumber', 'VehicleType', 'TrafficConditionNearby']
Date/time:
['LastUpdatedDate', 'LastUpdatedTime']


In [4]:
# @title Encode Columns

# Initiallise the encoded df
df_enc = df.copy()

# LabelEncode SystemCodeNumber using native factorise method.
# It wouldn't make a difference to LabelEncode since all the graphs are separated based on the SystemCodeNumber and no patterns would be detected from it.
df_enc['SystemCodeNumber_encoded'] = pd.factorize(df.SystemCodeNumber)[0]
print(df_enc.nunique()) # validate if all categories have been assigned to different numbers
print(df_enc.SystemCodeNumber_encoded.value_counts())

# One-Hot Encoding for 'VehicleType'
df_enc = pd.get_dummies(df_enc, columns=['VehicleType'], prefix='Vehicle')

# Ordinal Encoding for 'TrafficConditionNearby'
traffic_order = {'low': 0, 'average': 1, 'high': 2}
df_enc['TrafficConditionNearby_encoded'] = df_enc['TrafficConditionNearby'].map(traffic_order)
print(df_enc.TrafficConditionNearby_encoded.value_counts())

# Drop original columns
df_enc.drop(['SystemCodeNumber', 'TrafficConditionNearby'], axis=1, inplace=True)
df_enc.head()


SystemCodeNumber              14
Capacity                      14
Latitude                      14
Longitude                     14
Occupancy                   2347
VehicleType                    4
TrafficConditionNearby         3
QueueLength                   16
IsSpecialDay                   2
LastUpdatedDate               73
LastUpdatedTime              275
SystemCodeNumber_encoded      14
dtype: int64
SystemCodeNumber_encoded
0     1312
1     1312
2     1312
3     1312
4     1312
5     1312
6     1312
7     1312
8     1312
9     1312
10    1312
11    1312
12    1312
13    1312
Name: count, dtype: int64
TrafficConditionNearby_encoded
0    7793
1    6438
2    4137
Name: count, dtype: int64


,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,SystemCodeNumber_encoded,Vehicle_bike,Vehicle_car,Vehicle_cycle,Vehicle_truck,TrafficConditionNearby_encoded
ID,,,,,,,,,,,,,,
0,577,26.144536,91.736172,61,1,0,04-10-2016,07:59:00,0,False,True,False,False,0
1,577,26.144536,91.736172,64,1,0,04-10-2016,08:25:00,0,False,True,False,False,0
2,577,26.144536,91.736172,80,2,0,04-10-2016,08:59:00,0,False,True,False,False,0
3,577,26.144536,91.736172,107,2,0,04-10-2016,09:32:00,0,False,True,False,False,0
4,577,26.144536,91.736172,150,2,0,04-10-2016,09:59:00,0,True,False,False,False,0


In [5]:
df = df_enc.copy() # set the original df to the encoded df
df

,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,SystemCodeNumber_encoded,Vehicle_bike,Vehicle_car,Vehicle_cycle,Vehicle_truck,TrafficConditionNearby_encoded
ID,,,,,,,,,,,,,,
0,577,26.144536,91.736172,61,1,0,04-10-2016,07:59:00,0,False,True,False,False,0
1,577,26.144536,91.736172,64,1,0,04-10-2016,08:25:00,0,False,True,False,False,0
2,577,26.144536,91.736172,80,2,0,04-10-2016,08:59:00,0,False,True,False,False,0
3,577,26.144536,91.736172,107,2,0,04-10-2016,09:32:00,0,False,True,False,False,0
4,577,26.144536,91.736172,150,2,0,04-10-2016,09:59:00,0,True,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,1920,26.150504,91.733531,1517,6,0,19-12-2016,14:30:00,13,False,False,False,True,1
18364,1920,26.150504,91.733531,1487,3,0,19-12-2016,15:03:00,13,False,True,False,False,0
18365,1920,26.150504,91.733531,1432,3,0,19-12-2016,15:29:00,13,False,False,True,False,0


In [6]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

# Drop the original Date/time columns
df.drop(["LastUpdatedDate","LastUpdatedTime"], axis=1, inplace=True)

# 2. Pathway Pipeline Processing

In [7]:
# Save the selected columns to a CSV file for streaming or downstream processing
df.to_csv("parking_stream.csv", index=False)
df.head()

,Capacity,Latitude,Longitude,Occupancy,QueueLength,IsSpecialDay,SystemCodeNumber_encoded,Vehicle_bike,Vehicle_car,Vehicle_cycle,Vehicle_truck,TrafficConditionNearby_encoded,Timestamp
0,577,26.144536,91.736172,61,1,0,0,False,True,False,False,0,2016-10-04 07:59:00
1,1200,26.140014,91.731000,237,2,0,4,True,False,False,False,0,2016-10-04 07:59:00
2,687,20.000035,78.000003,264,2,0,3,False,True,False,False,0,2016-10-04 07:59:00
3,485,26.140048,91.730972,249,2,0,5,False,True,False,False,0,2016-10-04 07:59:00
4,1920,26.150504,91.733531,614,2,0,13,False,False,True,False,0,2016-10-04 07:59:00


In [8]:
# @title Pathway Schema
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Capacity: int    # Total parking capacity at the location
    Latitude: float  # Latitude of the location (may influence rates)
    Longitude: float # Longitude of the location (may influence rates)
    Occupancy: int   # Number of occupied parking spots
    QueueLength: int # Length of queue
    IsSpecialDay: int # If the day is a holiday
    SystemCodeNumber_encoded: int  # The encoded SystemCodeNumber
    Vehicle_bike: bool  # Encoded Vehicle column for bikes
    Vehicle_car: bool   # Encoded Vehicle column for cars
    Vehicle_cycle: bool # Encoded Vehicle column for cycles
    Vehicle_truck: bool # Encoded Vehicle column for trucks
    TrafficConditionNearby_encoded: int # Encoded column for the traffic condition
    Timestamp: str   # Timestamp of the observation


In [9]:
# @title Demo Replay
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv",
                          schema=ParkingSchema, input_rate=1312)

In [10]:
# @title Format Datetime
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# 3. The Pricing Functions

In [11]:
# @title Sample Pricing
# delta_window = (
#     data_with_time.windowby(
#         pw.this.t,  # Event time column to use for windowing (parsed datetime)
#         instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
#         window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
#         behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
#     )
#     .reduce(
#         t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
#         occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
#         occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
#         cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
#     )
#     .with_columns(
#         # Compute the price using a simple dynamic pricing formula:
#         #
#         # Pricing Formula:
#         #     price = base_price + demand_fluctuation
#         #     where:
#         #         base_price = 10 (fixed minimum price)
#         #         demand_fluctuation = (occ_max - occ_min) / cap
#         #
#         # Intuition:
#         # - The greater the difference between peak and low occupancy in a day,
#         #   the more volatile the demand is, indicating potential scarcity.
#         # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
#         # - This fluctuation is added to the base price of 10 to set the final price.
#         # - Example: If occ_max = 90, occ_min = 30, cap = 100
#         #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

#         price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
#     )
# )

In [12]:
# @title Baseline Pricing Model with cap/floor
import datetime

# 1. Modify Baseline Pricing Model
#    Remove demand-pressure logic. Replace with:
#    Pricet+1 = Pricet + alpha * (Occupancy / Capacity)
intermediate = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber_encoded, # To separate based on the system code number
        window=pw.temporal.tumbling(datetime.timedelta(days=1)), # Define a tumbling window with 1 day
                                                                # as the time interval to make the graph readable
        behavior=pw.temporal.exactly_once_behavior() # Exactly-once enable
    )
    .reduce(
        t=pw.this._pw_window_end,
        current_occupancy=pw.reducers.max(pw.this.Occupancy), # set occupancy
        current_capacity=pw.reducers.max(pw.this.Capacity), # Set capacity
        system_code=pw.reducers.max(pw.this.SystemCodeNumber_encoded) # set system code
    )
    .with_columns(
        occupancy_ratio=pw.apply(
            lambda occ, cap: occ / max(cap, 1), #calculate occupancy ratio as occupancy/capacity and 1 to avoid missing values for capacity
            pw.this.current_occupancy,
            pw.this.current_capacity
        )
    )
)

baseline_pricing = intermediate.with_columns(
    price=pw.apply(
        lambda r: max(5.0, min(20.0, 10.0 + 4.0 * r)), # apply the linear function with 5 to be min price and 20 to be max, where the equation is base price + 4*adjustment
        pw.this.occupancy_ratio
    )
)

In [13]:
# @title Demand-Based Pricing
#
# This code block defines a real-time, demand-based pricing model using the Pathway
# framework. The overall goal is to calculate a dynamic price for a service
# (e.g., parking, bike rental, toll road) based on various real-time factors.
#
# The model works in three main stages:
# 1. AGGREGATION: It processes a stream of raw data (`data_with_time`), grouping it
#    into daily summaries for each distinct system/location.
# 2. FEATURE ENGINEERING: It calculates a series of descriptive features from the
#    aggregated daily data. These features quantify concepts like occupancy,
#    congestion, and environmental factors.
# 3. PRICING CALCULATION: It combines these features into a final demand score,
#    which is then used to compute a smoothed, capped final price.
#
# The library `pw` refers to the Pathway library, which is designed for building
# real-time data processing pipelines that automatically update as new data arrives.

# --- STAGE 1: AGGREGATE REAL-TIME DATA INTO DAILY SUMMARIES ---
#
# We start by transforming the raw, high-frequency data stream into a more
# manageable, aggregated table. We want one summary row per system, per day.
demand_model = (
    # `windowby` is a powerful Pathway operation for time-series aggregation. It's
    # conceptually similar to a `GROUP BY` clause in SQL but specialized for time.
    data_with_time.windowby(
        # The first argument, `pw.this.t`, specifies the timestamp column (`t`)
        # in the input table (`data_with_time`) that will be used for windowing.
        pw.this.t,
        # The `instance` parameter defines the key for grouping. Here, we group by
        # `SystemCodeNumber_encoded`. This means the daily aggregation will be
        # performed independently for each unique system (e.g., for each parking garage).
        instance=pw.this.SystemCodeNumber_encoded,
        # The `window` parameter defines the size and type of the time windows.
        # `pw.temporal.tumbling` creates fixed-size, non-overlapping windows.
        # `datetime.timedelta(days=1)` sets the size of each window to one day.
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        # `exactly_once_behavior` is a processing guarantee. It ensures that each
        # incoming data point is processed in exactly one window, which is the
        # standard and desired behavior for daily summaries.
        behavior=pw.temporal.exactly_once_behavior()
    )
    # The `reduce` operation defines how to collapse all the data points within
    # a single window (e.g., all data for one garage for one day) into a single
    # output row.
    .reduce(
        # We create a new timestamp column 't' for our aggregated table.
        # `_pw_window_end` is a special Pathway variable that holds the end time
        # of the current window (e.g., 2023-10-27 23:59:59.999999).
        t=pw.this._pw_window_end,
        # For the following features, we use `pw.reducers.max`. This captures the
        # peak value for each metric during the day, which is often the most
        # relevant signal for demand-based pricing.
        # Peak occupancy for the day.
        current_occupancy=pw.reducers.max(pw.this.Occupancy),
        # The capacity of the system (likely constant, but `max` is a safe way to get it).
        current_capacity=pw.reducers.max(pw.this.Capacity),
        # The longest queue length observed during the day.
        current_queue=pw.reducers.max(pw.this.QueueLength),
        # `IsSpecialDay` is likely a flag (0 or 1). `max` will be 1 if any point
        # during the day was marked as special (e.g., a holiday or event).
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        # These vehicle flags are also likely 0/1. `max` captures if this vehicle
        # type was observed at all during the window.
        vehicle_truck=pw.reducers.max(pw.this.Vehicle_truck),
        vehicle_car=pw.reducers.max(pw.this.Vehicle_car),
        vehicle_bike=pw.reducers.max(pw.this.Vehicle_bike),
        vehicle_cycle=pw.reducers.max(pw.this.Vehicle_cycle),
        # The worst (highest encoded value) traffic condition nearby during the day.
        traffic_condition=pw.reducers.max(pw.this.TrafficConditionNearby_encoded),
        # We carry forward the system ID. Since we grouped by it, `max` (or `min`
        # or `any`) will simply return the ID for that group.
        system_code=pw.reducers.max(pw.this.SystemCodeNumber_encoded)
    )
)

# --- STAGE 2: FEATURE ENGINEERING ---
#
# Now that we have daily aggregated data, we create a series of more
# sophisticated features. Each `with_columns` call adds a new column to our
# `demand_model` table. This is done sequentially.
# The `pw.apply` function allows us to apply a custom Python lambda function row-wise.

# Feature 1: Occupancy Ratio
# This normalizes the absolute occupancy into a percentage (0.0 to 1.0), which
# is a much more comparable metric across systems of different sizes.
demand_model = demand_model.with_columns(
    occupancy_ratio=pw.apply(
        # The lambda function divides occupancy by capacity.
        # `max(cap, 1)` is a safety measure to prevent division-by-zero errors
        # if a system's capacity is ever reported as 0.
        lambda occ, cap: occ / max(cap, 1),
        pw.this.current_occupancy,
        pw.this.current_capacity
    )
)

# Feature 2: Queue Pressure
# This models the non-linear impact of a waiting queue. The formula uses a
# scaled hyperbolic tangent (tanh) function.
# The effect of a queue is small at first, grows rapidly, and then plateaus,
# reflecting that the difference between a 20-car queue and a 25-car queue
# is less impactful than the difference between a 0-car and a 5-car queue.
demand_model = demand_model.with_columns(
    queue_pressure=pw.apply(
        # The `tanh(x)` function, (e^x - e^-x)/(e^x + e^-x), is equivalent to
        # (e^(2x) - 1)/(e^(2x) + 1). Here, `x` is `q/5.0`.
        # The output is a value between -1 and 1, but since queue `q` >= 0, it's
        # between 0 and 1. This is then scaled by 0.3 to limit its total
        # influence on the final demand score.
        lambda q: (math.exp(2 * q / 5.0) - 1) / (math.exp(2 * q / 5.0) + 1) * 0.3,
        pw.this.current_queue
    )
)

# Feature 3: Traffic Multiplier
# This creates a factor that increases the price based on nearby traffic conditions.
demand_model = demand_model.with_columns(
    traffic_multiplier=pw.apply(
        # A simple linear model: starts at a base of 1.0 (no effect). For each
        # unit of the encoded traffic condition, the multiplier increases.
        # The `/ 2.0` and `* 0.2` are scaling factors that control the sensitivity.
        lambda traffic: 1 + (traffic / 2.0) * 0.2,
        pw.this.traffic_condition
    )
)

# Feature 4: Vehicle Premium/Discount
# This calculates a price adjustment based on the type of vehicle.
demand_model = demand_model.with_columns(
    vehicle_premium=pw.apply(
        # A weighted sum. This logic applies a surcharge for trucks (+25%),
        # no adjustment for cars (baseline), and a discount for bikes (-15%)
        # and cycles (-20%). This can be used to incentivize smaller vehicles.
        lambda truck, car, bike, cycle:
            truck * 0.25 + car * 0.0 + bike * (-0.15) + cycle * (-0.20),
        pw.this.vehicle_truck, pw.this.vehicle_car,
        pw.this.vehicle_bike, pw.this.vehicle_cycle
    )
)

# Feature 5: Special Day Surge
# This adds a significant surcharge if the day is a holiday, event day, etc.
demand_model = demand_model.with_columns(
    special_day_surge=pw.apply(
        # If `is_special_day` is 1, this adds a 0.3 surge factor. If 0, it adds nothing.
        lambda special: special * 0.3,
        pw.this.is_special_day
    )
)

# Feature 6: Core Demand
# This converts the linear `occupancy_ratio` into a non-linear demand signal
# using a logistic sigmoid function.
demand_model = demand_model.with_columns(
    core_demand=pw.apply(
        # The sigmoid function `1 / (1 + e^-x)` maps any input to a range (0, 1).
        # `(ratio - 0.5)` centers the function, so 50% occupancy is the midpoint.
        # The `-6` coefficient makes the curve steep, meaning demand rises sharply
        # as occupancy goes from, e.g., 40% to 60%.
        lambda ratio: 1.0 / (1.0 + math.exp(-6 * (ratio - 0.5))),
        pw.this.occupancy_ratio
    )
)

# Feature 7: Total Demand Score
# This combines the primary demand factors into a single weighted score.
demand_model = demand_model.with_columns(
    total_demand=pw.apply(
        # This is a weighted average that defines the core logic of the model:
        # - Core Demand (occupancy): 50% weight
        # - Queue Pressure: 20% weight
        # - Special Day Surge: 15% weight
        # - Traffic Multiplier: 15% weight
        # These weights are key business parameters that can be tuned.
        lambda core, queue, special, traffic:
            core * 0.5 + queue * 0.2 + special * 0.15 + (traffic - 1) * 0.15,
        pw.this.core_demand, pw.this.queue_pressure,
        pw.this.special_day_surge, pw.this.traffic_multiplier
    )
)

# --- STAGE 3: PRICING CALCULATION ---
#
# Finally, we translate the abstract demand score into a concrete price.

# Step 3a: Calculate a Raw, Unbounded Price
# This step combines the total demand score with the vehicle premium and a base price.
demand_model = demand_model.with_columns(
    raw_price=pw.apply(
        # Base price is 10.0. This is then scaled up by the demand score and
        # the vehicle adjustment factor.
        lambda demand, vehicle:
            10.0 * (1 + demand) * (1 + vehicle),
        pw.this.total_demand, pw.this.vehicle_premium
    )
)

# Step 3b: Calculate the Final, Smoothed and Bounded Price
# This final operation ensures the price is within a desired range (e.g., $5 to $20)
# and avoids abrupt jumps. It uses another sigmoid function for smoothing.
demand_based_pricing = demand_model.with_columns(
    price=pw.apply(
        # The formula `MIN_PRICE + (PRICE_RANGE / (1 + e^-k(x - MIDPOINT)))`
        # creates a smooth price curve.
        # - `5.0`: This is the minimum price (floor).
        # - `15.0`: This is the dynamic range of the price (`20.0 - 5.0`). The
        #   maximum price will be 5.0 + 15.0 = 20.0 (ceiling).
        # - `(raw - 12.5)`: Centers the curve. A `raw_price` of 12.5 is the midpoint.
        # - `-2`: Controls the steepness of the price increase around the midpoint.
        lambda raw:
            5.0 + 15.0 / (1.0 + math.exp(-2 * (raw - 12.5))),
        pw.this.raw_price
    )
)


In [14]:
# @title Competitive (optional) - Single plot
import datetime
import pathway as pw

# Model 3: Competitive Pricing Model
competitive_pricing = (
    data_with_time.windowby(
        pw.this.t,  # Event time column (parsed datetime)
        instance=pw.this.day,  # One instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # 1-hour windows
        behavior=pw.temporal.exactly_once_behavior()  # Exactly-once processing
    )
    .reduce(
        t=pw.this._pw_window_end,                           # Window end timestamp
        current_occupancy=pw.reducers.max(pw.this.Occupancy),
        current_capacity=pw.reducers.max(pw.this.Capacity),
        current_queue=pw.reducers.max(pw.this.QueueLength),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_truck=pw.reducers.max(pw.this.Vehicle_truck),
        vehicle_car=pw.reducers.max(pw.this.Vehicle_car),
        vehicle_bike=pw.reducers.max(pw.this.Vehicle_bike),
        vehicle_cycle=pw.reducers.max(pw.this.Vehicle_cycle),
        traffic_condition=pw.reducers.max(pw.this.TrafficConditionNearby_encoded),
        latitude=pw.reducers.max(pw.this.Latitude),
        longitude=pw.reducers.max(pw.this.Longitude),
        system_code=pw.reducers.max(pw.this.SystemCodeNumber_encoded),
        # Competitor data for nearby lots (simplified - getting averages)
        avg_competitor_occupancy=pw.reducers.avg(pw.this.Occupancy),
        avg_competitor_capacity=pw.reducers.avg(pw.this.Capacity),
    )
    .with_columns(
        # Step 1: Calculate occupancy ratio
        occupancy_ratio=pw.apply(
            lambda occ, cap: occ / max(cap, 1),
            pw.this.current_occupancy,
            pw.this.current_capacity
        )
    )
    .with_columns(
        # Step 2: Calculate queue pressure using tanh
        queue_pressure=pw.apply(
            lambda q: (pow(2.718281828, 2*q/5.0) - 1) / (pow(2.718281828, 2*q/5.0) + 1) * 0.3,
            pw.this.current_queue
        )
    )
    .with_columns(
        # Step 3: Calculate traffic multiplier
        traffic_multiplier=pw.apply(
            lambda traffic: 1 + (traffic / 2.0) * 0.2,
            pw.this.traffic_condition
        )
    )
    .with_columns(
        # Step 4: Calculate vehicle premium
        vehicle_premium=pw.apply(
            lambda truck, car, bike, cycle: truck * 0.25 + car * 0.0 + bike * (-0.15) + cycle * (-0.20),
            pw.this.vehicle_truck,
            pw.this.vehicle_car,
            pw.this.vehicle_bike,
            pw.this.vehicle_cycle
        )
    )
    .with_columns(
        # Step 5: Calculate special day surge
        special_day_surge=pw.apply(
            lambda special: special * 0.3,
            pw.this.is_special_day
        )
    )
    .with_columns(
        # Step 6: Calculate core demand using sigmoid
        core_demand=pw.apply(
            lambda ratio: 1.0 / (1.0 + pow(2.718281828, -6 * (ratio - 0.5))),
            pw.this.occupancy_ratio
        )
    )
    .with_columns(
        # Step 7: Calculate total demand (reduced weight for core_demand to make room for competitive factors)
        total_demand=pw.apply(
            lambda core, queue, special, traffic: core * 0.4 + queue * 0.2 + special * 0.15 + (traffic - 1) * 0.15,
            pw.this.core_demand,
            pw.this.queue_pressure,
            pw.this.special_day_surge,
            pw.this.traffic_multiplier
        )
    )
    .with_columns(
        # Step 8: Estimate competitor price based on regional average
        # competitor_occupancy_ratio = avg_competitor_occupancy / avg_competitor_capacity
        competitor_occupancy_ratio=pw.apply(
            lambda avg_occ, avg_cap: avg_occ / max(avg_cap, 1),
            pw.this.avg_competitor_occupancy,
            pw.this.avg_competitor_capacity
        )
    )
    .with_columns(
        # Step 9: Calculate estimated competitor price
        # competitor_price = base_price * (1 + competitor_occupancy_ratio * 0.5)
        estimated_competitor_price=pw.apply(
            lambda comp_ratio: 10.0 * (1 + comp_ratio * 0.5),
            pw.this.competitor_occupancy_ratio
        )
    )
    .with_columns(
        # Step 10: Calculate base competitive price (before competitive adjustment)
        base_competitive_price=pw.apply(
            lambda demand, vehicle: 10.0 * (1 + demand) * (1 + vehicle),
            pw.this.total_demand,
            pw.this.vehicle_premium
        )
    )
    .with_columns(
        # Step 11: Calculate competitive factor
        # If occupancy > 80%: premium pricing vs competitors
        # If occupancy <= 80%: slightly undercut competitors
        competitive_factor=pw.apply(
            lambda occ_ratio, base_price, comp_price:
                (0.95 if comp_price < base_price else 1.05) if occ_ratio > 0.8
                else (0.98 if comp_price > base_price else 1.02),
            pw.this.occupancy_ratio,
            pw.this.base_competitive_price,
            pw.this.estimated_competitor_price
        )
    )
    .with_columns(
        # Step 12: Apply competitive adjustment
        competitive_price=pw.apply(
            lambda base_price, factor: base_price * factor,
            pw.this.base_competitive_price,
            pw.this.competitive_factor
        )
    )
    .with_columns(
        # Step 13: Determine dynamic max price based on market pressure
        # market_pressure = occupancy_ratio * queue_pressure
        market_pressure=pw.apply(
            lambda occ_ratio, queue: occ_ratio * queue,
            pw.this.occupancy_ratio,
            pw.this.queue_pressure
        )
    )
    .with_columns(
        # Step 14: Calculate final bounded price
        # High market pressure allows higher max price (22), otherwise 18
        price=pw.apply(
            lambda comp_price, market_press:
                max(4.0, min(22.0 if market_press > 0.7 else 18.0, comp_price)),
            pw.this.competitive_price,
            pw.this.market_pressure
        )
    )
)


In [19]:
# @title Plot Initiallisation
import bokeh.plotting
import panel as pn
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure

# Single Graph Price plotter
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# 2. Filter and Plot One Graph per SystemCodeNumber
plot_dict = {}
valid_codes = df['SystemCodeNumber_encoded'].unique()  # avoid blanks for missing codes
for code in valid_codes:
    print(f"Generating plot for SystemCodeNumber_encoded = {code}")
    filtered = baseline_pricing.filter(pw.this.system_code == code)
    viz = filtered.plot(price_plotter, sorting_col="t")
    plot_dict[code] = viz

import bokeh.plotting
import bokeh.models
import bokeh.layouts

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
# 4. Use `pn.Column(*plot_dict.values()).servable()` to display all plots
# layout = pn.Column(*plot_dict.values())
# layout.servable()

Generating plot for SystemCodeNumber_encoded = 0


Generating plot for SystemCodeNumber_encoded = 4
Generating plot for SystemCodeNumber_encoded = 3


Generating plot for SystemCodeNumber_encoded = 5
Generating plot for SystemCodeNumber_encoded = 13
Generating plot for SystemCodeNumber_encoded = 2


Generating plot for SystemCodeNumber_encoded = 6
Generating plot for SystemCodeNumber_encoded = 11
Generating plot for SystemCodeNumber_encoded = 7


Generating plot for SystemCodeNumber_encoded = 8
Generating plot for SystemCodeNumber_encoded = 1
Generating plot for SystemCodeNumber_encoded = 9
Generating plot for SystemCodeNumber_encoded = 10
Generating plot for SystemCodeNumber_encoded = 12


In [20]:
# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order

# Model 2: Demand-based pricing plots (corrected to use demand_based_pricing instead of demand_model)
plot_dict_demand_corrected = {}
valid_codes = df['SystemCodeNumber_encoded'].unique()

for code in valid_codes:
    print(f"Generating demand-based plot for SystemCodeNumber_encoded = {code}")
    filtered = demand_based_pricing.filter(pw.this.system_code == code)
    viz = filtered.plot(price_plotter, sorting_col="t")
    plot_dict_demand_corrected[code] = viz

# Create layout for demand-based pricing plots
layout_demand = pn.Column(*plot_dict_demand_corrected.values())
layout_demand.servable()

import bokeh.plotting
import bokeh.models
import bokeh.layouts

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
# 4. Use `pn.Column(*plot_dict_demand.values()).servable()` to display all plots
layout_demand = pn.Column(*plot_dict_demand_corrected.values())
layout_demand.servable()



Generating demand-based plot for SystemCodeNumber_encoded = 0
Generating demand-based plot for SystemCodeNumber_encoded = 4


Generating demand-based plot for SystemCodeNumber_encoded = 3


Generating demand-based plot for SystemCodeNumber_encoded = 5
Generating demand-based plot for SystemCodeNumber_encoded = 13


Generating demand-based plot for SystemCodeNumber_encoded = 2
Generating demand-based plot for SystemCodeNumber_encoded = 6


Generating demand-based plot for SystemCodeNumber_encoded = 11
Generating demand-based plot for SystemCodeNumber_encoded = 7


Generating demand-based plot for SystemCodeNumber_encoded = 8
Generating demand-based plot for SystemCodeNumber_encoded = 1


Generating demand-based plot for SystemCodeNumber_encoded = 9
Generating demand-based plot for SystemCodeNumber_encoded = 10
Generating demand-based plot for SystemCodeNumber_encoded = 12


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [28]:
# --- Comparison Plots for Model 1 (Baseline) vs. Model 2 (Demand-Based) ---
print("\n--- Generating comparison plots for Model 1 vs. Model 2 ---")

# Step 1: Rename the price columns in each model's table to avoid conflicts during the join
model1_renamed = baseline_pricing.with_columns(price_model1=pw.this.price)
model2_renamed = demand_based_pricing.with_columns(price_model2=pw.this.price)

# Step 2: Select only the necessary columns for a clean join
model1_to_join = model1_renamed.select(pw.this.t, pw.this.system_code, pw.this.price_model1)
model2_to_join = model2_renamed.select(pw.this.t, pw.this.system_code, pw.this.price_model2)

# Step 3: Join the two pricing models using proper Pathway join syntax
combined_pricing = model1_to_join.join(
    model2_to_join,
    pw.left.t == pw.right.t,
    pw.left.system_code == pw.right.system_code
).select(
    t=pw.this.t,
    system_code=pw.this.system_code,
    price_model1=pw.this.price_model1,
    price_model2=pw.right.price_model2  # Access the right table's column explicitly
)

# Step 4: Define a plotting function that can draw two lines on the same figure
def combined_plotter(source, system_code_id):
    """
    Bokeh plotting function to compare Model 1 and Model 2 prices on a single graph.
    """
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=f"Price Comparison: Model 1 vs. Model 2 (System Code: {system_code_id})",
        x_axis_type="datetime"
    )
    # Plot Model 1 (Baseline Price)
    fig.line("t", "price_model1", source=source, line_width=2, color="blue", legend_label="Model 1 (Baseline)")

    # Plot Model 2 (Demand-Based Price)
    fig.line("t", "price_model2", source=source, line_width=2, color="green", legend_label="Model 2 (Demand-Based)")

    fig.legend.location = "top_left"
    fig.yaxis.axis_label = "Price"
    fig.xaxis.axis_label = "Time"

    return fig

# Step 5: Create a dictionary to hold the comparison plots
comparison_plots = {}
valid_codes = df['SystemCodeNumber_encoded'].unique()

# Step 6: Loop through each system code, filter the joined data, and generate a plot
for code in valid_codes:
    # Filter the combined data for the current system code
    filtered_combined = combined_pricing.filter(pw.this.system_code == code)

    # Create a plot function instance that includes the system code for the title
    plot_function = lambda source, code=code: combined_plotter(source, system_code_id=code)

    # Generate the plot
    viz = filtered_combined.plot(plot_function, sorting_col="t")
    comparison_plots[code] = viz

# Step 7: Create a Panel layout to display all comparison plots
layout_comparison = pn.Column(*comparison_plots.values(), name="Model 1 vs. Model 2 Comparison")
print(f"Generated {len(comparison_plots)} comparison plots.")

# To display the plots, you would use:
layout_comparison.servable()



--- Generating comparison plots for Model 1 vs. Model 2 ---
Generated 14 comparison plots.


Column(name='Model 1 vs. M...)
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [31]:
# --- Visualizing Competitor Price from Model 3 ---
print("\n--- Generating plot for Competitor Prices (Model 3) ---")

def competitor_price_plotter(source):
    """
    Bokeh plotting function for visualizing the estimated average competitor price.
    """
    fig = bokeh.plotting.figure(
        height=450,
        width=850,
        title="Model 3: Estimated Average Competitor Parking Price Over Time",
        x_axis_type="datetime",
    )
    # Plot the estimated competitor price over time
    fig.line("t", "estimated_competitor_price", source=source, line_width=2, color="purple", legend_label="Avg. Competitor Price")
    fig.circle("t", "estimated_competitor_price", source=source, size=6, color="red")
    fig.yaxis.axis_label = "Estimated Price"
    fig.xaxis.axis_label = "Time"
    fig.legend.location = "top_left"
    return fig

# Use Pathway's .plot() method to bind the competitive_pricing stream to the plotter
# This will create a single, live-updating plot.
competitor_plot = competitive_pricing.plot(competitor_price_plotter, sorting_col="t")

# Create a Panel layout to display the plot
layout_model3 = pn.Column(competitor_plot, name="Model 3: Competitor Pricing Analysis")
print("Generated plot for Model 3 competitor prices.")

# To display the plot, you would use:
layout_model3.servable()



--- Generating plot for Competitor Prices (Model 3) ---
Generated plot for Model 3 competitor prices.


Column(name='Model 3: Competitor P...)
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

 4. Run

In [32]:
# %%capture --no-display
pw.run()

Output()